# Caso 1: Optimización de Mezcla de Combustibles

## Maestría en Inteligencia Artificial y Analítica de Datos (MIAAD)
**Alumno:** Javier Augusto Rebull Saucedo (al263483)

### Introducción al Problema
Una refinería necesita producir **95 barriles** de gasolina mezclando dos tipos de combustibles (A y B). El objetivo es minimizar el costo total, cumpliendo con restricciones de calidad (octanaje), ambientales (emisiones de CO₂) y límites de producción. Crucialmente, solo se pueden usar barriles enteros.

**Datos del Problema:**

| Combustible | Costo ($/barril) | Octanaje | Emisiones CO₂ (kg/barril) | Límite de Producción |
| :---: | :---: | :---: | :---: | :---: |
| A | 50 | 92 | 3.0 | 70 barriles |
| B | 35 | 70 | 2.1 | 80 barriles |

**Requisitos de la Mezcla Final:**
- Octanaje Promedio Mínimo: 80.
- Emisiones Totales Máximas: 275 kg de CO₂.

In [1]:
#Instalacion
!pip install pulp

#Configuración e Instalación de Librerías
import pulp
import pandas as pd
print("Librería PuLP importada correctamente.")

Librería PuLP importada correctamente.


In [2]:
#Definición de Datos y Parámetros

# Datos de los combustibles organizados en un diccionario
data = {
    'A': {'costo': 50, 'octanaje': 92, 'emisiones': 3.0, 'limite_max': 70},
    'B': {'costo': 35, 'octanaje': 70, 'emisiones': 2.1, 'limite_max': 80}
}
combustibles = data.keys()

# Requerimientos de la mezcla final
produccion_requerida = 95
octanaje_minimo = 80
emisiones_maximas = 275

## Formulación del Modelo Matemático

Implementamos el modelo de Programación Lineal Entera (ILP).

In [3]:
# 1. Inicializar el modelo (Minimización de costos)
model = pulp.LpProblem("Mezcla_Combustibles_Costo_Minimo", pulp.LpMinimize)

# 2. Definir las Variables de Decisión
# X_A, X_B: Número de barriles de A y B.
# Deben ser enteras (cat='Integer') y no negativas (lowBound=0).
X = pulp.LpVariable.dicts("Barriles", combustibles, lowBound=0, cat='Integer')

# 3. Definir la Función Objetivo
# Minimizar Z = 50*X_A + 35*X_B
model += pulp.lpSum([data[i]['costo'] * X[i] for i in combustibles]), "Costo_Total"

In [4]:
# 4. Definir las Restricciones

# R1: Producción Total (X_A + X_B = 95)
model += pulp.lpSum([X[i] for i in combustibles]) == produccion_requerida, "Produccion_Total"

# R2: Octanaje Mínimo (92*X_A + 70*X_B >= 80 * 95)
# Se requiere linealizar la restricción de promedio. 80 * 95 = 7600.
octanaje_requerido_total = octanaje_minimo * produccion_requerida
model += pulp.lpSum([data[i]['octanaje'] * X[i] for i in combustibles]) >= octanaje_requerido_total, "Octanaje_Minimo"

# R3: Límite de Emisiones (3.0*X_A + 2.1*X_B <= 275)
model += pulp.lpSum([data[i]['emisiones'] * X[i] for i in combustibles]) <= emisiones_maximas, "Emisiones_CO2"

# R4: Límites de Producción Individuales (X_A <= 70, X_B <= 80)
for i in combustibles:
    model += X[i] <= data[i]['limite_max'], f"Limite_Max_{i}"

print("Modelo formulado correctamente.")
# print(model) # Descomentar para ver la formulación completa

Modelo formulado correctamente.


In [5]:
# Resolución del Modelo

# 5. Resolver el modelo
print("Iniciando resolución...")
model.solve()

# Verificar el estado de la solución
estado = pulp.LpStatus[model.status]
print(f"Estado de la solución: {estado}")

Iniciando resolución...
Estado de la solución: Optimal


In [6]:
# Interpretación y Presentación de Resultados

# 6. Interpretar y mostrar los resultados
if estado == 'Optimal':
    print("\n" + "="*40)
    print("  SOLUCIÓN ÓPTIMA ENCONTRADA")
    print("="*40)

    # Mostrar el Costo Mínimo
    print(f"Costo Mínimo Total: ${pulp.value(model.objective):,.2f}")

    # Mostrar la mezcla óptima
    print("\n--- Plan de Mezcla Óptimo ---")
    for i in combustibles:
        print(f" - Combustible {i}: {X[i].varValue:.0f} barriles")

    # Verificación de métricas clave
    print("\n--- Verificación de Métricas ---")
    barriles_A = X['A'].varValue
    barriles_B = X['B'].varValue

    # Verificar que los valores se recuperaron correctamente
    if barriles_A is not None and barriles_B is not None:
        # Octanaje Promedio
        octanaje_final = (data['A']['octanaje'] * barriles_A + data['B']['octanaje'] * barriles_B) / produccion_requerida
        print(f" - Octanaje Promedio: {octanaje_final:.2f} (Mínimo requerido: {octanaje_minimo})")

        # Emisiones Totales
        emisiones_finales = (data['A']['emisiones'] * barriles_A + data['B']['emisiones'] * barriles_B)
        print(f" - Emisiones CO2: {emisiones_finales:.1f} kg (Máximo permitido: {emisiones_maximas} kg)")
    else:
        print("Error al recuperar los valores de las variables.")

    print("="*40)

else:
    print(f"No se encontró una solución óptima. Estado: {estado}. Revisar las restricciones del modelo.")


  SOLUCIÓN ÓPTIMA ENCONTRADA
Costo Mínimo Total: $3,985.00

--- Plan de Mezcla Óptimo ---
 - Combustible A: 44 barriles
 - Combustible B: 51 barriles

--- Verificación de Métricas ---
 - Octanaje Promedio: 80.19 (Mínimo requerido: 80)
 - Emisiones CO2: 239.1 kg (Máximo permitido: 275 kg)


## Decisión Prescrita

El modelo de optimización ILP prescribe la siguiente estrategia para minimizar los costos:

- **Usar 44 barriles de Combustible A.**
- **Usar 51 barriles de Combustible B.**

El costo mínimo total es de **$3,985.00**.

**Análisis:** La solución muestra que la restricción de octanaje es el factor limitante. Para minimizar el costo, el modelo busca maximizar el uso del Combustible B (más barato). Sin embargo, necesita al menos 44 barriles del Combustible A (más caro, pero de alto octanaje) para alcanzar el promedio mínimo requerido de 80. El octanaje final es 80.19, muy cercano al límite.